#Midterm

**Name : Uma Srikanth Reddy Koduru***



Uid: U94125452

Importing the pandas and cassandra driver

In [55]:
import pandas as pd

In [56]:
from cassandra.cluster import Cluster

Creating a cluster of the Cassandra Database

In [57]:
cluster = Cluster()

In [58]:
session = cluster.connect()

Keyspace is like a database and the below code drops the keyspace if it already exists

In [59]:
session.execute('''DROP KEYSPACE IF EXISTS company''')

Creating the Keyspace company

Rationale: the below keyspace is named company for easy understanding and the replication factor is set to 3, that means the data is replicated 3 times, thus the data is disaster relilient.

In [60]:
session.execute('''CREATE KEYSPACE company WITH REPLICATION = {
                'class' : 'SimpleStrategy',
                'replication_factor' : 3
}''')

Creating a table chatApplication inside the keyspace

Rationale: The table name is put as chatApplicaton for easy understanding and the columns - user_id is Integer type as id's are generally integers, Date is stored as Text as the DATE format did not work, file_type is Text as it describes whether the file is json or vedio or an image, The file columns is stored as Text as the BLOB type did not work

The primary keys are chosen as user_id and date. In cassandra the hasing is done using the user_id to store the data in the nodes, so if felt user_id of INT data type is a natural fit.

In [61]:
session.execute('''CREATE TABLE IF NOT EXISTS company.chatApplication(
                user_id INT,
                date TEXT,
                file_type TEXT,
                file TEXT,
                PRIMARY KEY(user_id, date)
)''')

In [62]:
session.execute('''DROP TABLE company.chatApplication''')

Inserting the data into the data frame data with with  user_id, date, file_type and file as collumns

In [63]:
data = [
    (1, '10/22/2023', 'JPEG', 'IMAGE'),
    (2, '11/22/2023', 'MP3', 'VEDIO'),
    (3, '12/22/2023', 'JSON', 'FILE')
]

df = pd.DataFrame(data, columns=['user_id', 'date', 'file_type', 'file'])

Visualizing the data with the help of a data frame

In [64]:
df

,user_id,date,file_type,file
0,1,10/22/2023,JPEG,IMAGE
1,2,11/22/2023,MP3,VEDIO
2,3,12/22/2023,JSON,FILE


The below add_records function insert the above data into the table chatApplication

In [65]:
def add_records():
    for row in data:
        session.execute(
            """
            INSERT INTO company.chatApplication (user_id, date, file_type, file)
            VALUES (%s, %s, %s, %s)
            """,
            (row[0], row[1], row[2], row[3])
    )

The below function search_records() implemets the query for searching the records of user_id = 1

In [66]:
def search_records():
    rows = session.execute("select (user_id, date, file_type, file) from company.chatApplication where  user_id= 1;")
    for j in rows:
        print(f"{j[0][0]}, {j[0][1]}, {j[0][2]}, {j[0][3]}")

The below delete_records() function removes the record for the user_id =1

In [67]:
def delete_records():
    session.execute('''FROM company.chatApplication
                    DELETE date
                    WHERE user_id  = 1''')

In [68]:
add_records()

InvalidRequest: Error from server: code=2200 [Invalid query] message="table chatapplication does not exist"

In [69]:
search_records()

InvalidRequest: Error from server: code=2200 [Invalid query] message="table chatapplication does not exist"

In [ ]:
delete_records()